In [1]:
import ee
ee.Initialize()

In [29]:
import geemap
Map = geemap.Map()

In [15]:
def testfeature(feature):
    num = ee.Number.parse(feature.get('areasqkm'))
    return feature.set('areasqkm', num)

# Load watersheds from a data table.
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06').\
filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29)).\
map(testfeature)    
    

    
#  Display the table and print its first element.
#Map.addLayer(sheds, {}, 'watersheds');
#print('First watershed', sheds.first());

#  Print the number of watersheds.
#print('Count:', sheds.size());

#  Print stats for an area property.
#print('Area stats:', sheds.aggregate_stats('areasqkm'));

In [10]:
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06')

In [16]:
largeSheds = sheds.filter(ee.Filter.gt('areasqkm',25000))

In [25]:
def addArea(feature):
    return feature.set({'area' : feature.geometry().area().divide(100*100)})

In [26]:
dd = sheds.map(addArea)

In [28]:
dd.first().propertyNames().getInfo()

['area',
 'sourcedata',
 'areaacres',
 'sourcefeat',
 'shape_leng',
 'states',
 'sourceorig',
 'huc6',
 'areasqkm',
 'shape_area',
 'tnmid',
 'name',
 'metasource',
 'loaddate',
 'gnis_id',
 'system:index']

In [38]:
def getCentroid(feature):
    keepProperties = ['name', 'huc6', 'tnmid', 'areasqkm']
    centroid = feature.geometry().centroid()
    return ee.Feature(centroid).copyProperties(feature, keepProperties)

In [39]:
centroids = sheds.map(getCentroid)

In [44]:
sheds = ee.FeatureCollection('USGS/WBD/2017/HUC06').\
filterBounds(ee.Geometry.Rectangle(-127.18, 19.39, -62.75, 51.29));

In [45]:
def areaDiff(feature):
    area = feature.geometry().area().divide(1000*1000)
    diff = area.subtract(ee.Number.parse(feature.get('areasqkm')))
    return feature.set('diff',diff.pow(2))

In [49]:
rmse = ee.Number(sheds.map(areaDiff).\
reduceColumns(ee.Reducer.mean(),['diff']).\
get('mean')).sqrt()

In [51]:
rmse.getInfo()

50.29671055754766

In [42]:
Map.addLayer(centroids,{'color': 'FF0000'},'centroids')

In [35]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…